In [1]:
import numpy as np
import networkx as nx
import pandas as pd
import anndata as ad
from anndata import AnnData
import scanpy as sc


In [2]:
adata_exp0 = ad.read_h5ad('C:/Users/mdichgan/Documents/Helmholtz/send_to_Jakob/spatial/counts_CPc_exp0_BA28.h5ad')

In [53]:
adata_Yao = ad.read_h5ad(
    'C:/Users/mdichgan/Documents/Helmholtz/send_to_Jakob/sc/Yao_150kcells_subsample_with_annotations_sparse_subset.h5ad')

In [26]:
adata_Yao.obs["celltype"] = adata_Yao.obs["label"]

In [21]:
sc.pp.normalize_total(adata_exp0)
sc.pp.normalize_total(adata_Yao)
# sc.pp.log1p(adata_exp0)
# sc.pp.log1p(adata_Yao)
adata_exp0.layers["lognorm"] = adata_exp0.X
adata_Yao.layers["lognorm"] = adata_Yao.X

In [30]:
def get_modality_ratio_score(adata_st: AnnData, adata_sc: AnnData, obs_key: str = "celltype", k: int = 45,ct_filter_factor: float = 2):
    adata_st.obs["modality"] = "spatial"
    adata_sc.obs["modality"] = "sc"
    adata = ad.concat([adata_st, adata_sc], join='inner')  

    # Set counts to log norm data
    adata.X = adata.layers["lognorm"]
    
    # Calculate PCA (Note: we could also think about pca per cell type...)
    assert (adata.obsm is None) or ('X_pca' not in adata.obsm), "PCA already exists."
    sc.tl.pca(adata)
    
    # get cell type groups
    sc_cts = set(adata_sc.obs["celltype"].cat.categories)
    st_cts = set(adata_st.obs["celltype"].cat.categories)
    shared_cts = list(sc_cts.intersection(st_cts))         

    # Get ratio per shared cell type
    df = pd.DataFrame(columns=["cell_id","celltype","ratio","exp_val","score"])
    for ct in shared_cts:
        enough_cells = (adata.obs.loc[adata.obs[obs_key]==ct,"modality"].value_counts() > (ct_filter_factor * k)).all()     #nochmal: wieso ct_fil?
        if enough_cells:
            a = adata[adata.obs[obs_key]==ct]
            exp_val = (a.obs.loc[a.obs["modality"]=="sc"].shape[0])/a.obs.shape[0]
            sc.pp.neighbors(a,n_neighbors=k)
            G = nx.Graph(incoming_graph_data=a.obsp["connectivities"])
            nx.set_node_attributes(G, {i:a.obs["modality"].values[i] for i in range(G.number_of_nodes())}, "modality")   

            ct_df = np.zeros(a.obs.shape[0])
            f = np.vectorize(lambda x: x/exp_val if x>0 and x<=exp_val else x/(exp_val-1)+1/(1-exp_val))
            i = 0
            for cell in G.nodes():
                ct_df[i] = sum(1 for neighbor in G.neighbors(cell) if G.nodes[cell]["modality"]=="sc")  #number_modality_sc
                ct_df[i] = ct_df[i]/G.degree(cell)  
                i += 1
                  #ratio: number modality sc / total cells
                # if total_cells != 0:              #0 nicht möglich? immer k?
                #     ratio = number_modality_sc / total_cells
                # else:
                #     ratio = 0        #no neighbors
                #df.append({"cell_id": "cell_id", "celltype": ct, "ratio": ratio, "exp_val":exp_val, "score": score})  #wie machen mit cell_id? ineffizient
                #df aus liste an listen
            daf = pd.DataFrame({"cell_id": a.obs["cell_id"], "celltype": ct, "ratio":ct_df, "exp_val":exp_val, "score":f(ct_df)})
    
    df = pd.concat([df,daf])
    return df


In [54]:
adata_Yao.obs = adata_Yao.obs.reset_index().rename(columns={'index':'cell_id'})
adata_Yao.obs

,cell_id,sample_name,Unnamed: 0,cl,exp_component_name,donor_label,sex_label,region_label,gene.counts,library_label,...,ss_cluster_id,ss_cluster_color,ss_cluster_label,tenx_cluster_id,tenx_cluster_color,tenx_cluster_label,cell_type_accession_id,cell_type_accession_color,cell_type_accession_label,sss
0,1,10X_cells.AAACCTGCAAGCTGTT-L8TX_180221_01_F09,279041,182,AAACCTGCAAGCTGTT-L8TX_180221_01_F09,371230,M,ACA,4043,SM-DPC4J-01,...,0,NaN,absent,104,#B1EC30,186_L2 IT RSP-ACA,134,#B1EC30,CCN19103010000134,205694
1,2,10X_cells.AAACCTGCACGTCTCT-L8TX_180221_01_F09,279043,6,AAACCTGCACGTCTCT-L8TX_180221_01_F09,371230,M,ACA,4408,SM-DPC4J-01,...,0,NaN,absent,9,#935F50,3_Lamp5 Lhx6,6,#9E7153,CCN19103010000006,205696
2,11,10X_cells.AAAGCAACAGCTGCAC-L8TX_180221_01_F09,279075,179,AAAGCAACAGCTGCAC-L8TX_180221_01_F09,371230,M,ACA,3832,SM-DPC4J-01,...,0,NaN,absent,139,#02F970,183_L2/3 IT CTX_1,168,#02F970,CCN19103010000168,205728
3,21,10X_cells.AAATGCCGTAATCACC-L8TX_180221_01_F09,279106,60,AAATGCCGTAATCACC-L8TX_180221_01_F09,371230,M,ACA,2908,SM-DPC4J-01,...,0,NaN,absent,48,#CD6DF2,56_Vip_2,60,#DA81F7,CCN19103010000060,205759
4,28,10X_cells.AACCATGGTGATGTCT-L8TX_180221_01_F09,279133,201,AACCATGGTGATGTCT-L8TX_180221_01_F09,371230,M,ACA,5333,SM-DPC4J-01,...,0,NaN,absent,153,#55C5B5,205_L5 IT CTX_1,204,#4E9E9E,CCN19103010000204,205786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24504,149919,10X_cells.TGCCAAAGTACATGTC-L8TX_200611_02_A05,204240,46,TGCCAAAGTACATGTC-L8TX_200611_02_A05,395345,F,TEa-PERI-ECT,2790,SM-G9JI2-5,...,0,NaN,absent,39,#D26AE6,45_Vip_2,45,#D26AE6,CCN19103010000045,130893
24505,149922,10X_cells.TGCGCAGAGTGACTCT-L8TX_200611_02_A05,204272,369,TGCGCAGAGTGACTCT-L8TX_200611_02_A05,395345,F,TEa-PERI-ECT,1775,SM-G9JI2-5,...,0,NaN,absent,309,#2E3E39,359_OPC_1,365,#2E3E39,CCN19103010000365,130925
24506,149923,10X_cells.TGCTGCTGTACTCGCG-L8TX_200611_02_A05,204300,332,TGCTGCTGTACTCGCG-L8TX_200611_02_A05,395345,F,TEa-PERI-ECT,5464,SM-G9JI2-5,...,0,NaN,absent,277,#2E8CCB,327_L6 CT CTX,282,#296B98,CCN19103010000282,130953
24507,149936,10X_cells.TTAGGCACAGACGTAG-L8TX_200611_02_A05,140873,359,TTAGGCACAGACGTAG-L8TX_200611_02_A05,395345,F,TEa-PERI-ECT,6460,SM-G9JI2-5,...,0,NaN,absent,286,#573D90,346_L6b CTX,314,#71419D,CCN19103010000314,67526


In [31]:
#test get_modality_ratio_score
get_modality_ratio_score(adata_exp0, adata_Yao)       

C:\Users\mdichgan\AppData\Local\Temp\ipykernel_6204\229719651.py:25: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  sc.pp.neighbors(a,n_neighbors=k)


KeyError: 'cell_id'